<a href="https://colab.research.google.com/github/pratikunterwegs/holeybirds/blob/master/get_elev_indices_hula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!/usr/bin/env python
# coding: utf-8

import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [10]:
# print where we are
os.getcwd()
cmcp_shp = '../data/spatial/mask_cov_layers_wgs84.shp'
# read local shp to gee
cmcp = geemap.shp_to_ee(cmcp_shp)

In [11]:
# show on map
Map = geemap.Map()
Map.addLayer(cmcp, {}, 'cmcp')
## Map # uncomment to see map

In [27]:
# define start and end date june and dec 2016
start_date = '2016-06-01'
end_date = '2016-10-01'

# get sentinel data for the time range 2016
sentinel = ee.ImageCollection("COPERNICUS/S2")
sentinel_bounds = sentinel.filterBounds(cmcp)
sentinel_2016 = sentinel_bounds.filterDate(start_date, end_date)

In [28]:
# adding a NDVI band
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
    return image.addBands([ndvi])

In [29]:
# map metrics over sentinel
sentinel_ndvi = sentinel_2016.map(add_ndvi)

# get median values
ndvi_median = sentinel_ndvi.select('ndvi').median()

# clip to geometry
ndvi_clip = ndvi_median.clip(cmcp)

In [ ]:
# Define an sld style color ramp to apply to the image.
sld_ramp = \
  '<RasterSymbolizer>' + \
    '<ColorMap type="ramp" extended="false" >' + \
      '<ColorMapEntry color="#0D0887" quantity="-1" label="0"/>' + \
      '<ColorMapEntry color="#CC4678" quantity="0" label="50" />' + \
      '<ColorMapEntry color="#F0F921" quantity="0.5" label="100" />' + \
    '</ColorMap>' + \
  '</RasterSymbolizer>';


# print layers
vis = {'bands': ['ndvi']}
Map = geemap.Map(center=[33.0005,33.5], zoom=9)
Map.addLayer(ndvi_clip.sldStyle(sld_ramp), {}, 'ndvi')
Map.addLayer(cmcp)
Map.addLayerControl()

Map

In [31]:
# export ndvi data as an image locally
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'raster_hula_clip_wgs84.tif')

geemap.ee_export_image(
    ndvi_clip, filename=filename, scale=10, region=cmcp.geometry(),
    file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to /home/pratik/Downloads/raster_hula_clip_wgs84.tif
